<a href="https://colab.research.google.com/github/AltafParekh2001/GEN_AI_Project-s/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -------------------------------
# 📦 Install Required Packages
# -------------------------------
# !pip install --upgrade transformers torch gradio pandas matplotlib openpyxl

# -------------------------------
# 📑 Import Required Libraries
# -------------------------------
import re
import torch
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from transformers import pipeline




In [2]:

# -------------------------------
# 🚀 Load Sentiment Analysis Pipeline
# -------------------------------
# Loads a pre-trained sentiment-analysis pipeline from Hugging Face Transformers
pipe = pipeline(
    "text-classification",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)

# -------------------------------
# 🧮 Text Sentiment Prediction
# -------------------------------

def sentiment_analysis(text):
    """
    Predict sentiment of input text.
    Returns: sentiment label and confidence score.
    """
    result = pipe(text)
    return result[0]['label'], float(result[0]['score'])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:

# -------------------------------
# 📂 File Upload Handler & Batch Processing
# -------------------------------

def read_review(file_obj):
    """
    Reads reviews from an Excel file and predicts sentiment for each review.
    Returns: DataFrame with Sentiments and a bar chart
    """
    # Read Excel file into pandas DataFrame
    df = pd.read_excel(file_obj)

    # Ensure 'Review' column exists
    if 'Review' not in df.columns:
        raise ValueError("Excel file must contain a 'Review' column")

    # Run sentiment analysis on each review (this returns a tuple)
    sentiments = df['Review'].apply(sentiment_analysis)
    df['Sentiment'] = sentiments.apply(lambda x: x[0])
    df['Confidence'] = sentiments.apply(lambda x: x[1])

    # Chart object
    chart_obj = sentiment_bar_chart(df)
    return df, chart_obj


In [4]:
# -------------------------------
# 📊 Sentiment Distribution Plot
# -------------------------------

def sentiment_bar_chart(df):
    """
    Returns a matplotlib bar chart showing count of each sentiment label.
    """
    counts = df['Sentiment'].value_counts()
    fig, ax = plt.subplots()
    counts.plot(kind='bar', ax=ax, color=['#4caf50', '#f44336'])
    ax.set_xlabel('Sentiment')
    ax.set_ylabel('Count')
    ax.set_title('Sentiment Distribution')
    return fig


In [5]:
# -------------------------------
# 🧑‍💻 Try with a Single Example
# -------------------------------

# Example: Test the pipeline with a simple string
example_text = "I am sad today"
sentiment, score = sentiment_analysis(example_text)
print(f"Text: {example_text}\nSentiment: {sentiment}, Score: {score:.4f}")



Text: I am sad today
Sentiment: NEGATIVE, Score: 0.9986


In [6]:
# -------------------------------
# ✨ Gradio Interface
# -------------------------------

demo = gr.Interface(
    fn=read_review,
    inputs=gr.File(file_types=['.xlsx'], label="Upload your review Excel file (.xlsx only, must have a 'Review' column)"),
    outputs=[
        gr.Dataframe(label="Review Results (with Sentiment & Confidence)"),
        gr.Plot(label="Sentiment Distribution Bar Chart")
    ],
    title="Batch Sentiment Analysis",
    description="Upload an Excel file with a column 'Review' to analyze sentiments at scale."
)

# Uncomment to launch interface directly in a notebook or script:
demo.launch()

# -------------------------------
# ℹ️ Notes:
# - The input Excel file MUST have a column 'Review' with text review strings.
# - The pipeline/plotting works for two-class (POSITIVE/NEGATIVE) analysis.
# -------------------------------


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38df0c337ad5d7374c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
